### Imports

In [7]:

import networkx as nx
import numpy as np
import random, math
import graphviz
import functools, collections, operator
import seaborn as sns

from scipy.stats import shapiro
import scipy.stats as scs
from scipy.stats import describe

import plotly.figure_factory as ff

from scipy.stats import norm
import scipy.stats as stats
from scipy.stats import pearsonr

import os
import time

import pylab

from statsmodels.graphics.gofplots import qqplot

#import kaleido

import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import plotly.graph_objects as go
from pyvis.network import Network
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.express as px
from plotly._subplots import make_subplots
import plotly

import statsmodels.formula.api as sm

from stargazer.stargazer import Stargazer


# import models
from classes.jackson_model import JacksonSimulationV2
from classes.network_drawing import plotly_sim_drawing
from classes.plotly_drawings import plotly_lines


def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


### Showcases

In [12]:

# showcases
showcase_sim_no_bias = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=False, rho=1, epsilon=1)
showcase_sim_expo = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1, epsilon=1.75)
showcase_sim_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.75, epsilon=1)
showcase_sim_expo_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.5, epsilon=1.5)

showcase_sims = [showcase_sim_no_bias, showcase_sim_expo, showcase_sim_fb, showcase_sim_expo_fb]

showcase_plots = [plotly_sim_drawing.plotly_draw(plotly_sim_drawing(), showcase_sim, layout='spring', draw_largest_CC=True, title=None) for showcase_sim in showcase_sims]





### Rho and Epsilon distributions

In [29]:

'''DESCRIPTIVE STATISTICS'''
df_eps_rho_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county', 'Epsilon', 'EpsilonHigh', 'Rho', 'RhoHigh']].dropna()

df_eps_rho_chetty = df_eps_rho_chetty.apply(pd.to_numeric, errors='ignore')


df_chetty_summ = df_eps_rho_chetty[['Epsilon', 'Rho']].describe().round(4)
#df_chetty_summ.loc['skew'] = df_chetty_summ.skew()
#df_chetty_summ.to_latex('data.tex')

#Stargazer(df_chetty_summ)
#df_eps_rho_chetty.skew()

eps_all = list(df_eps_rho_chetty['Epsilon'].values) + list(df_eps_rho_chetty['EpsilonHigh'].values)
rho_all = list(df_eps_rho_chetty['Rho'].values) + list(df_eps_rho_chetty['RhoHigh'].values)

print(pearsonr(eps_all, rho_all))

df_eps_rho_chetty[['Epsilon', 'Rho', 'EpsilonHigh', 'RhoHigh']].corr()

np.mean(rho_all)

df_eps_rho_chetty.describe()


PearsonRResult(statistic=0.3270609836794669, pvalue=3.5985301884551014e-150)


,county,exposure_grp_mem_county,exposure_grp_mem_high_county,bias_grp_mem_county,bias_grp_mem_high_county,Epsilon,EpsilonHigh,Rho,RhoHigh
count,3012.000000,3012.000000,3012.000000,3012.000000,3012.000000,3012.000000,3012.000000,3012.000000,3012.000000
mean,30206.501328,0.906089,1.078581,0.064161,-0.187968,1.354583,0.932459,1.131563,0.661774
std,15139.364060,0.211814,0.209551,0.050633,0.064644,0.675489,0.416315,0.113567,0.052487
min,1001.000000,0.255200,0.510130,-0.108090,-0.536180,0.345641,0.200365,0.796738,0.499872
25%,18166.500000,0.765533,0.931867,0.030808,-0.221698,0.899858,0.629484,1.059508,0.625343
50%,29116.000000,0.905695,1.081650,0.063970,-0.179075,1.208249,0.849027,1.125496,0.657728
75%,45037.500000,1.052710,1.227383,0.095450,-0.143335,1.612561,1.146228,1.192859,0.692894
max,56045.000000,1.486280,1.666160,0.334570,-0.043250,6.836991,2.920569,2.529641,0.930369


In [3]:

'''FIGURES'''

# import the data, convert
df_eps_rho_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county', 'Epsilon', 'EpsilonHigh', 'Rho', 'RhoHigh']].dropna()
df_eps_rho_chetty = df_eps_rho_chetty.apply(pd.to_numeric, errors='ignore')
df_eps_rho_chetty["EpsilonColor"] = np.where(df_eps_rho_chetty["Epsilon"]<1, 'red', 'blue')
df_eps_rho_chetty["RhoColor"] = np.where(df_eps_rho_chetty["Rho"]<1, 'red', 'blue')


vars = ['Epsilon', 'Rho']# 'EpsilonHigh', 'Rho', 'RhoHigh']
figs = []

for var in vars:
    # Winsorize at the 0.5% level
    df_eps_rho_chetty = df_eps_rho_chetty[(df_eps_rho_chetty[var] <= df_eps_rho_chetty[var].quantile(0.995)) & df_eps_rho_chetty[var] >= df_eps_rho_chetty[var].quantile(0.005)]

    #calculate mean and variance
    mu, std = norm.fit(df_eps_rho_chetty[var])

    # Shapiro Wilk test (want p<0.05)
    stat, p = shapiro(df_eps_rho_chetty[var])

    # count share below 1
    below_one_share = len([x for x in list(df_eps_rho_chetty[var]) if x < 1]) / len(list(df_eps_rho_chetty[var]))

    fig_title = f'{var} density plot<br>Winsorised at 0.5% level<br>Normal distribution: mean={round(mu,3)}, se={round(std, 3)}<br>Shapiro-Wilk stat and prob: {format(stat, ".3f")}, {"{:.1e}".format(p)}<br>Prob <1: {round(below_one_share, 3)}'

    # Create a histogram of the column
    fig = px.histogram(df_eps_rho_chetty, x=var, nbins=100, opacity=0.9, marginal=None, histnorm='probability density', title=fig_title, width=600, height=600)#, color=df_eps_rho_chetty[var + 'Color'])

    # Add a normal distribution curve to the histogram
    x_axis = np.linspace(df_eps_rho_chetty[var].min(), df_eps_rho_chetty[var].max(), 100)
    y_axis = norm.pdf(x_axis, mu, std)
    curve = go.Scatter(x=x_axis, y=y_axis, mode='lines', line=dict(width=2, color='black'), name=f'Normal Distribution: ', showlegend=False)
    fig.add_trace(curve)

    figs.append(fig)

for var in vars:
    stat, p = shapiro(df_eps_rho_chetty[var])
    print(f'{var} has a SW stat of {format(stat, ".3f")} with p-value {"{:.1e}".format(p)}')




Epsilon has a SW stat of 0.901 with p-value 2.6e-40
Rho has a SW stat of 0.992 with p-value 5.1e-12


# New stuff

### General simulation code

In [3]:

# runs a lot of MTO simulations for a givne list of epsilon and rho values, returning a df
def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


#df_test = MTO_simulator(list_rho_values=[1,2], list_eps_values=[1,2], num_MTO_per_sim=100, num_sim_per_param=5)
#df_test.sample(5)


### Epsilon effect on MTO

In [20]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.01)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_eps = MTO_simulator(list_rho_values=[1, 1.25, 1.5], list_eps_values=list_eps_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps.sample(5)
#df_eps.to_csv('eps_sim.csv')

df_eps = pd.read_csv('SimulationData/eps_sim.csv')
df_eps.sample(5)



,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
106154,bldKBovX,0.5,0.85,1.25,0.230769,0.461538,0.923077,14,13,0.032058,0.458220,0.431050,22.42,0.273586,0.432432,1.081081,0.103030
416007,snxbiIlt,0.5,1.88,1.50,0.153846,0.428571,0.948718,14,13,0.422161,0.763697,0.323370,21.04,0.275803,0.231481,0.694444,0.246073
71895,pwRtAKJI,0.5,0.73,1.50,0.214286,0.428571,0.928571,14,14,0.076316,0.385028,0.306018,20.64,0.261309,0.385356,1.156069,0.174224
403060,dBDDFyoI,0.5,1.84,1.25,0.454545,1.142857,0.943182,14,11,0.327096,0.700388,0.354626,22.02,0.284262,0.281690,0.704225,0.139394
106424,ZcWDAUYc,0.5,0.85,1.25,0.363636,1.000000,0.948052,14,11,0.002612,0.516520,0.497920,22.14,0.274465,0.432432,1.081081,0.103030


In [29]:

'''
heatmap = px.density_heatmap(x=df_eps['epsilon'], y=df_eps['H_Share'], range_x=[0.5,2], range_y=[0,1])
lines = px.line(x=df_eps['epsilon'], y=df_eps['Predicted_H_Share'], range_x=[0.5,2], range_y=[0,1])

fig = go.Figure(data = heatmap.data + lines.data)
fig.show()
'''

plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1], x_var='epsilon', y_var='Network_H_Share_L', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(255,0,0,1)', 'rgba(0,255,0,1)'])




### Rho effect on MTO

In [16]:

# general simulation stuff
list_rho_values = np.arange(1, 2, 0.01)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_rho = MTO_simulator(list_eps_values=[1, 1.25, 1.5], list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_rho.to_csv('rho_sim.csv')

df_rho = pd.read_csv('rho_sim.csv')

df_rho.sample(5)

,Unnamed: 0,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
220616,220616,PSBqoYwx,0.5,1.50,1.20,0.363636,0.923077,0.939394,14,11,0.184696,0.562626,0.363968,22.42,0.273072,0.333333,0.800000,0.107143
168220,168220,GTrzcAKI,0.5,1.25,1.68,0.307692,0.714286,0.938462,14,13,0.237305,0.556455,0.304101,19.92,0.257679,0.264550,0.888889,0.274194
239658,239658,liGVDWHx,0.5,1.50,1.39,0.214286,0.428571,0.928571,14,14,0.277930,0.700215,0.410837,21.58,0.279944,0.287770,0.800000,0.189627
93530,93530,bWauSXDu,0.5,1.00,1.93,0.333333,1.142857,0.958333,14,9,0.296085,0.642491,0.334876,18.52,0.244299,0.259067,1.000000,0.317406
274179,274179,JbKpvGNw,0.5,1.50,1.74,0.454545,1.142857,0.943182,14,11,0.383628,0.730444,0.330923,20.30,0.261135,0.229885,0.800000,0.307479


In [20]:

'''
heatmap = px.density_heatmap(x=df_rho['rho'], y=df_rho['H_Share'], range_x=[1,2], range_y=[0,1])
lines = px.line(x=df_rho['rho'], y=df_rho['Predicted_H_Share'], range_x=[1,2], range_y=[0,1])

fig = go.Figure(data = heatmap.data + lines.data)
fig.show()
'''

rho_exposure = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho, x_var='rho', y_var='Network_H_Share_H', x_range=[1,2], y_range=[0,2], x_title='rho', y_title='Exposure', with_predicted=False)
#MTO_sim_results(df_results[df_results['rho']==1], 'epsilon', 'H_Share', [1,2], [0,1.1], 'epsilon', 'Exposure')



## Both rho and epsilon

### Specified values

In [7]:

df_rho_eps = pd.read_csv('big_boy.csv').drop(columns='Unnamed: 0')
df_results = df_rho_eps.copy()



,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
0,cAoqpRJp,0.5,0.50,1.00,0.736842,1.473684,0.947368,20,19,0.666667,1.333333,0.000000
1,cAoqpRJp,0.5,0.50,1.00,0.789474,1.578947,0.947368,20,19,0.666667,1.333333,0.000000
2,cAoqpRJp,0.5,0.50,1.00,0.750000,1.500000,0.950000,20,20,0.666667,1.333333,0.000000
3,cAoqpRJp,0.5,0.50,1.00,0.550000,1.100000,0.950000,20,20,0.666667,1.333333,0.000000
4,cAoqpRJp,0.5,0.50,1.00,0.684211,1.368421,0.947368,20,19,0.666667,1.333333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
599995,WtQLTSHa,0.5,1.95,1.95,0.153846,0.842105,0.980769,20,13,0.173837,0.677966,0.385737
599996,WtQLTSHa,0.5,1.95,1.95,0.157895,0.315789,0.947368,20,19,0.173837,0.677966,0.385737
599997,WtQLTSHa,0.5,1.95,1.95,0.187500,0.631579,0.968750,20,16,0.173837,0.677966,0.385737
599998,WtQLTSHa,0.5,1.95,1.95,0.400000,1.052632,0.960000,20,15,0.173837,0.677966,0.385737


In [3]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.05)
list_rho_values = np.arange(1, 2, 0.05)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network

#df_rho_eps = MTO_simulator(list_eps_values=list_eps_values, list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_rho_eps.to_csv('eps_rho_sim.csv')

df_eps_rho = pd.read_csv('eps_rho_sim.csv')

df_eps_rho.sample(5)


,Unnamed: 0,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
265257,265257,cFdMBOjt,0.5,1.15,1.25,0.300000,0.727273,0.925000,14,10,0.171130,0.589578,0.418362,22.06,0.273633,0.372093,0.930233,0.117949
423264,423264,RNAezYzN,0.5,1.55,1.15,0.461538,1.000000,0.934066,14,13,0.241334,0.739027,0.483330,22.92,0.291239,0.341006,0.784314,0.083558
482901,482901,FiPNFYLF,0.5,1.70,1.10,0.538462,1.142857,0.932692,14,13,0.261222,0.710414,0.435419,23.30,0.290880,0.336700,0.740741,0.059233
36624,36624,rKciBaYr,0.5,0.55,1.80,0.400000,1.142857,0.950000,14,10,0.048099,0.575553,0.520033,17.80,0.223039,0.358423,1.290323,0.221106
11693,11693,qNBVggss,0.5,0.50,1.55,0.545455,1.285714,0.939394,14,11,-0.072921,0.542384,0.619172,19.00,0.248911,0.430108,1.333333,0.154930


### Share of H friends: redo this and the rho and only vary rho or epsilon (with more values)

In [23]:

fig1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='epsilon', y_var='Network_H_Share_H', x_range=[1,2], y_range=[0,2], x_title='epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(255,0,0,1)'])
fig2 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='epsilon', y_var='Network_H_Share_L', x_range=[1,2], y_range=[0,2], x_title='epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(0,0,255,1)'])

fig1['data'][0]['name'] = 'High SES'
fig2['data'][0]['name'] = 'Low SES'

fig = go.Figure(data = fig2.data + fig1.data)

fig.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Share of HSES friends',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')


In [24]:



fig1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_H_Share_H', x_range=[1,2], y_range=[0,2], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(255,0,0,1)'])
fig2 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_H_Share_L', x_range=[1,2], y_range=[0,2], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(0,0,255,1)'])

fig1['data'][0]['name'] = 'High SES'
fig2['data'][0]['name'] = 'Low SES'

fig = go.Figure(data = fig2.data + fig1.data)

fig.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'Share of HSES friends',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [1,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig.add_vline(x=1, line_width=1, annotation_text = 'rho=1: no bias')




### SES Assortativity

In [25]:

fig_eps = plotly_lines.line_with_error(plotly_lines(), data=df_eps_rho, x_var='epsilon', y_var='Network_SES_assortativity', color='rgba(100,0,100,1)', legend_text='')

fig_eps.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = False, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

#fig_eps.data[0].line.color = "rgb(0,100,0,1)"
fig_eps.add_hline(y=0, line_width=1, annotation_text = 'Random mixing')
fig_eps.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')




In [26]:

fig_rho = plotly_lines.line_with_error(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_SES_assortativity', color='rgba(100,0,100,1)', legend_text='')

fig_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = False, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [1,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_rho.add_hline(y=0, line_width=1, annotation_text = 'Random mixing')
fig_rho.add_vline(x=1, line_width=1, annotation_text = 'ρ=1: no bias')




### MTO Outcomes

In [6]:


fig_exposure = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Exposure'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.RdYlBu_r, range_color=(0.6,1.4))

fig_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [1,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_exposure.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias')
fig_exposure.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias')




NameError: name 'df_eps_rho' is not defined

In [7]:

px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Friend_Bias'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.RdBu_r)

fig_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [1,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_exposure.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias')
fig_exposure.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias')


NameError: name 'df_eps_rho' is not defined

In [8]:
px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['H_Share'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.Greens)

NameError: name 'df_eps_rho' is not defined

### Randomly sampled values

In [2]:

list_eps_values_random = np.random.normal(loc=1.1, scale=0.3, size=[10])
list_rho_values_random = np.random.normal(loc=1.1, scale=0.1, size=[10])

num_sim_per_param = 1 # number of simulations with given parameter set
num_MTO_per_sim = 50 # number of simulations on each network

df_rho_eps = MTO_simulator(list_eps_values=list_eps_values_random, list_rho_values=list_rho_values_random, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
df_rho_eps.sample(5)


Running 5000 simulations over 100 parameter combinations
 Finished 5000 simulations in 1.69 minutes, average time: 0.000338 min/sim

,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
3938,PioXsrhM,0.5,1.146198,1.018463,0.615385,1.230769,0.923077,14,13,0.024510,0.434923,0.409328,24.16,0.295267,0.457494,0.931880,0.009764
4861,UgeUzwMb,0.5,1.168129,0.950575,0.230769,0.461538,0.923077,14,13,0.064136,0.525026,0.459174,24.42,0.292484,0.485209,0.922454,-0.027357
3201,kqlPdREJ,0.5,1.108535,1.041340,0.357143,0.714286,0.928571,14,14,0.067729,0.510370,0.441027,23.92,0.292147,0.455435,0.948526,0.021272
537,nHajByjJ,0.5,0.717291,1.027146,0.571429,1.142857,0.928571,14,14,-0.139457,0.466883,0.611711,24.08,0.290593,0.566923,1.164625,0.011212
3752,mUkidFhH,0.5,1.146198,1.139185,0.500000,1.000000,0.928571,14,14,0.094537,0.625529,0.524474,23.10,0.283844,0.409012,0.931880,0.069190


### Descriptive statistics

How does the variance change with rho and epsilon?

Would appear that variance in Exposure and Friend bias increases some, as does degree. Vriance in H_Share might be going down(although need to test thsi)

I can run regressions on thisssss

### Regressions

In [3]:

df_rho_eps.groupby(['epsilon', 'rho']).var()
#df_test = df_results[['Predicted_H_Share', 'H_Share', 'Predicted_Exposure', 'Exposure', 'Predicted_Friend_Bias', 'Friend_Bias', 'Degree', 'N_exposure']].describe()



p_SES_high   H_Share  Exposure  Friend_Bias  N_exposure  \
epsilon  rho                                                                 
0.585438 0.950575         0.0  0.009206  0.036823     0.000011         0.0   
         0.979175         0.0  0.017794  0.071175     0.000023         0.0   
         1.018463         0.0  0.016220  0.064143     0.000014         0.0   
         1.027146         0.0  0.012896  0.053308     0.000016         0.0   
         1.041340         0.0  0.014763  0.054338     0.000026         0.0   
...                       ...       ...       ...          ...         ...   
1.445301 1.098516         0.0  0.013270  0.056260     0.000046         0.0   
         1.101921         0.0  0.013851  0.062114     0.000038         0.0   
         1.139185         0.0  0.021214  0.079809     0.000050         0.0   
         1.143984         0.0  0.013919  0.060916     0.000048         0.0   
         1.156695         0.0  0.021858  0.090642     0.000064         0.0   

                     Degree  Network_SES_assortativity  Network_H_Share_H  \
epsilon  rho                                                                
0.585438 0.950575  0.336327                        0.0                0.0   
         0.979175  0.630204                        0.0                0.0   
         1.018463  0.455510                        0.0                0.0   
         1.027146  0.622449                        0.0                0.0   
         1.041340  0.654694                        0.0                0.0   
...                     ...                        ...                ...   
1.445301 1.098516  0.762857                        0.0                0.0   
         1.101921  0.803673                        0.0                0.0   
         1.139185  0.640408                        0.0                0.0   
         1.143984  1.112245                        0.0                0.0   
         1.156695  0.877143                        0.0                0.0   

                   Network_H_Share_L  Network_Degree  Network_Clustering  \
epsilon  rho                                                               
0.585438 0.950575                0.0             0.0                 0.0   
         0.979175                0.0             0.0                 0.0   
         1.018463                0.0             0.0                 0.0   
         1.027146                0.0             0.0                 0.0   
         1.041340                0.0             0.0                 0.0   
...                              ...             ...                 ...   
1.445301 1.098516                0.0             0.0                 0.0   
         1.101921                0.0             0.0                 0.0   
         1.139185                0.0             0.0                 0.0   
         1.143984                0.0             0.0                 0.0   
         1.156695                0.0             0.0                 0.0   

                   Predicted_H_Share  Predicted_Exposure  \
epsilon  rho                                               
0.585438 0.950575                0.0                 0.0   
         0.979175                0.0                 0.0   
         1.018463                0.0                 0.0   
         1.027146                0.0                 0.0   
         1.041340                0.0                 0.0   
...                              ...                 ...   
1.445301 1.098516                0.0                 0.0   
         1.101921                0.0                 0.0   
         1.139185                0.0                 0.0   
         1.143984                0.0                 0.0   
         1.156695                0.0                 0.0   

                   Predicted_Friend_Bias  
epsilon  rho                              
0.585438 0.950575                    0.0  
         0.979175                    0.0  
         1.018463                    0.0  
         1.027146

In [4]:

result = sm.ols(formula="H_Share ~ epsilon + rho", data=df_rho_eps).fit()
print(result.params)
print(result.summary())


Intercept    0.777015
epsilon     -0.220436
rho         -0.059284
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                H_Share   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     517.2
Date:                Mon, 01 May 2023   Prob (F-statistic):          7.02e-205
Time:                        11:45:45   Log-Likelihood:                 2855.5
No. Observations:                5000   AIC:                            -5705.
Df Residuals:                    4997   BIC:                            -5685.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

### Predicted versus realised

In [5]:

result = sm.ols(formula="H_Share ~ Predicted_H_Share", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept            0.108410
Predicted_H_Share    0.797058
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                H_Share   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     912.2
Date:                Mon, 01 May 2023   Prob (F-statistic):          3.23e-184
Time:                        11:45:57   Log-Likelihood:                 2804.2
No. Observations:                5000   AIC:                            -5604.
Df Residuals:                    4998   BIC:                            -5591.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

In [5]:

result = sm.ols(formula="Exposure ~ Predicted_Exposure", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept             0.249063
Predicted_Exposure    0.749443
dtype: float64
                            OLS Regression Results                            
Dep. Variable:               Exposure   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     1302.
Date:                Sat, 29 Apr 2023   Prob (F-statistic):          2.41e-271
Time:                        20:17:36   Log-Likelihood:                -1407.3
No. Observations:               12500   AIC:                             2819.
Df Residuals:                   12498   BIC:                             2834.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [6]:

result = sm.ols(formula="Friend_Bias ~ Predicted_Friend_Bias", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept                0.925382
Predicted_Friend_Bias    0.089281
dtype: float64
                            OLS Regression Results                            
Dep. Variable:            Friend_Bias   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     339.3
Date:                Sat, 29 Apr 2023   Prob (F-statistic):           8.68e-75
Time:                        20:17:40   Log-Likelihood:                 31285.
No. Observations:               12500   AIC:                        -6.257e+04
Df Residuals:                   12498   BIC:                        -6.255e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

### Sample rho and epsilon from the distribution, run MTO 100 times per simulation, and vary w_H: this shouild hopefully show how important it is to consider friendingf bias 